# Comparative Analysis of Air Quality Variability Between Major and Peripheral Cities: A Case Study of Thailand

## Import libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, mannwhitneyu, shapiro, levene
from itertools import product

## Import dataset

In [2]:
df = pd.read_csv("thai_air_data_cleaned.csv")

In [3]:
df.head()

,Timestamp,City,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time
0,2025-02-19 18:11:33,Samut Prakan,"City Hall, Samut Prakan, Thailand (ศาลากลางจัง...",13.599149,100.597345,0.1,-5.9,8.4,9.3,13.4,1008.5,55.0,110.0,0.6,0.6,31.7,0.5,2025-02-20 00:00:00
1,2025-02-19 18:11:33,Nakhon Ratchasima,"Municipal Waste Water Pumping Station, Nakhon ...",14.979537,102.098335,NaN,-4.6,9.2,11.0,23.7,985.9,NaN,100.0,0.2,0.6,31.9,0.5,2025-02-20 00:00:00
2,2025-02-19 18:11:34,Surin,"Pyramid House, Surin, Thailand",14.793612,103.430560,NaN,-4.6,9.2,NaN,NaN,993.9,NaN,NaN,NaN,NaN,32.0,0.5,NaN
3,2025-02-19 18:11:34,Sakon Nakhon,"Meteorological stations, Sakon Nakhon, Thailan...",17.156620,104.133216,0.1,-4.9,9.9,NaN,20.1,991.0,43.0,63.0,NaN,1.1,30.2,1.5,2025-02-20 00:00:00
4,2025-02-19 18:11:35,Mueang Khon Kaen,"Khonkaen, Thailand (ส่วนอุทกวิทยา สำนักงานทรัพ...",16.445383,102.835250,NaN,-5.2,9.0,NaN,NaN,988.0,NaN,79.0,0.6,NaN,31.5,0.5,2025-02-20 00:00:00


In [4]:
df.shape

(2146, 18)

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2146 entries, 0 to 2145
Data columns (total 18 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Timestamp  2146 non-null   object 
 1   City       2146 non-null   object 
 2   name       2146 non-null   object 
 3   lat        2146 non-null   float64
 4   lon        2146 non-null   float64
 5   co         1375 non-null   float64
 6   dew        1811 non-null   float64
 7   h          2146 non-null   float64
 8   no2        1655 non-null   float64
 9   o3         1699 non-null   float64
 10  p          2146 non-null   float64
 11  pm10       1836 non-null   float64
 12  pm25       2084 non-null   float64
 13  r          1141 non-null   float64
 14  so2        1561 non-null   float64
 15  t          2146 non-null   float64
 16  w          2146 non-null   float64
 17  time       2084 non-null   object 
dtypes: float64(14), object(4)
memory usage: 301.9+ KB


In [6]:
df["time"] = pd.to_datetime(df["time"])

In [7]:
df.isna().sum()

Timestamp       0
City            0
name            0
lat             0
lon             0
co            771
dew           335
h               0
no2           491
o3            447
p               0
pm10          310
pm25           62
r            1005
so2           585
t               0
w               0
time           62
dtype: int64

In [8]:
(df.groupby(["City"]).apply(lambda x: x.isna().sum() / len(x) * 100, include_groups=False)
                     .drop(columns="City", errors="ignore").round(2))

,Timestamp,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time
City,,,,,,,,,,,,,,,,,
Ayutthaya,0.0,0.0,0.0,0.0,0.00,14.75,0.0,0.0,0.00,0.0,0.00,0.0,54.10,0.00,0.0,0.0,0.0
Bangkok,0.0,0.0,0.0,0.0,0.00,100.00,0.0,0.0,0.00,0.0,0.00,0.0,21.31,0.00,0.0,0.0,0.0
Chiang Mai,0.0,0.0,0.0,0.0,100.00,100.00,0.0,0.0,0.00,0.0,0.00,0.0,63.93,0.00,0.0,0.0,0.0
Chonburi,0.0,0.0,0.0,0.0,100.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,31.67,0.00,0.0,0.0,0.0
Kamphaeng Phet,0.0,0.0,0.0,0.0,100.00,100.00,0.0,100.0,100.00,0.0,0.00,0.0,100.00,100.00,0.0,0.0,0.0
Kanchanaburi,0.0,0.0,0.0,0.0,0.00,0.00,0.0,100.0,0.00,0.0,0.00,0.0,34.92,100.00,0.0,0.0,0.0
Krabi,0.0,0.0,0.0,0.0,100.00,0.00,0.0,100.0,100.00,0.0,100.00,0.0,100.00,100.00,0.0,0.0,0.0
Lampang,0.0,0.0,0.0,0.0,0.00,18.03,0.0,0.0,0.00,0.0,0.00,0.0,47.54,0.00,0.0,0.0,0.0
Lamphun,0.0,0.0,0.0,0.0,100.00,0.00,0.0,0.0,0.00,0.0,0.00,0.0,75.81,0.00,0.0,0.0,0.0


In [9]:
df.groupby(["City"]).size()

City
Ayutthaya              61
Bangkok                61
Chiang Mai             61
Chonburi               60
Kamphaeng Phet          1
Kanchanaburi           63
Krabi                   1
Lampang                61
Lamphun                62
Mueang Chiang Rai      61
Mueang Khon Kaen       60
Nakhon Pathom          60
Nakhon Ratchasima      63
Nakhon Sawan           63
Nan                    62
Narathiwat             60
Nong Khai              63
Nonthaburi             62
Pathum Thani           60
Pattani                 1
Phayao                 62
Phitsanulok            62
Phuket                 60
Prachuap Khiri Khan    61
Ratchaburi             60
Rayong                 60
Sakon Nakhon           62
Samut Prakan           63
Samut Sakhon           61
Saraburi               60
Satun                  60
Suphan Buri            63
Surin                  62
Trang                  60
Trat                   61
Ubon Ratchathani       61
Uthai Thani             1
Uttaradit              61
Yala   

In [10]:
# remove cities with only one record
df = df[~df["City"].isin(["Kamphaeng Phet", "Krabi", "Pattani", "Uthai Thani", "Surin"])]

df.groupby(["City"]).size()

City
Ayutthaya              61
Bangkok                61
Chiang Mai             61
Chonburi               60
Kanchanaburi           63
Lampang                61
Lamphun                62
Mueang Chiang Rai      61
Mueang Khon Kaen       60
Nakhon Pathom          60
Nakhon Ratchasima      63
Nakhon Sawan           63
Nan                    62
Narathiwat             60
Nong Khai              63
Nonthaburi             62
Pathum Thani           60
Phayao                 62
Phitsanulok            62
Phuket                 60
Prachuap Khiri Khan    61
Ratchaburi             60
Rayong                 60
Sakon Nakhon           62
Samut Prakan           63
Samut Sakhon           61
Saraburi               60
Satun                  60
Suphan Buri            63
Trang                  60
Trat                   61
Ubon Ratchathani       61
Uttaradit              61
Yala                   60
dtype: int64

## Data preprocessing

### Define city type (major/peripheral) and region (North, Northeast, East, Central, West, South)

In [11]:
# City type: major and peripheral
city_type = {
    'Samut Prakan': 'Peripheral',
    'Nakhon Ratchasima': 'Major',
    'Surin': 'Peripheral',
    'Sakon Nakhon': 'Peripheral',
    'Mueang Khon Kaen': 'Major',
    'Nong Khai': 'Peripheral',
    'Kanchanaburi': 'Major',
    'Suphan Buri': 'Peripheral',
    'Uthai Thani': 'Peripheral',
    'Nakhon Sawan': 'Peripheral',
    'Phitsanulok': 'Peripheral',
    'Kamphaeng Phet': 'Peripheral',
    'Lamphun': 'Peripheral',
    'Lampang': 'Peripheral',
    'Phayao': 'Peripheral',
    'Bangkok': 'Major',
    'Trat': 'Peripheral',
    'Nonthaburi': 'Peripheral',
    'Nan': 'Peripheral',
    'Ubon Ratchathani': 'Peripheral',
    'Samut Sakhon': 'Peripheral',
    'Chiang Mai': 'Major',
    'Mueang Chiang Rai': 'Peripheral',
    'Ayutthaya': 'Peripheral',
    'Uttaradit': 'Peripheral',
    'Rayong': 'Major',
    'Pathum Thani': 'Peripheral',
    'Chonburi': 'Major',
    'Nakhon Pathom': 'Peripheral',
    'Ratchaburi': 'Peripheral',
    'Phuket': 'Major',
    'Yala': 'Peripheral',
    'Krabi': 'Peripheral',
    'Trang': 'Peripheral',
    'Satun': 'Peripheral',
    'Pattani': 'Peripheral',
    'Narathiwat': 'Peripheral',
    'Prachuap Khiri Khan': 'Peripheral',
    'Saraburi': 'Peripheral',
}

df['City_type'] = df['City'].map(city_type)

In [12]:
# Region: north, northeast, central, south, west, east
city_region = {
    'Samut Prakan': 'Central',
    'Nakhon Ratchasima': 'Northeast',
    'Surin': 'Northeast',
    'Sakon Nakhon': 'Northeast',
    'Mueang Khon Kaen': 'Northeast',
    'Nong Khai': 'Northeast',
    'Kanchanaburi': 'West',
    'Suphan Buri': 'Central',
    'Uthai Thani': 'Central',
    'Nakhon Sawan': 'Central',
    'Phitsanulok': 'Central',
    'Kamphaeng Phet': 'Central',
    'Lamphun': 'North',
    'Lampang': 'North',
    'Phayao': 'North',
    'Bangkok': 'Central',
    'Trat': 'East',
    'Nonthaburi': 'Central',
    'Nan': 'North',
    'Ubon Ratchathani': 'Northeast',
    'Samut Sakhon': 'Central',
    'Chiang Mai': 'North',
    'Mueang Chiang Rai': 'North',
    'Ayutthaya': 'Central',
    'Uttaradit': 'North',
    'Rayong': 'East',
    'Pathum Thani': 'Central',
    'Chonburi': 'East',
    'Nakhon Pathom': 'Central',
    'Ratchaburi': 'West',
    'Phuket': 'South',
    'Yala': 'South',
    'Krabi': 'South',
    'Trang': 'South',
    'Satun': 'South',
    'Pattani': 'South',
    'Narathiwat': 'South',
    'Prachuap Khiri Khan': 'West',
    'Saraburi': 'Central',
}

df['City_region'] = df['City'].map(city_region)

In [13]:
df.groupby(["City_region", "City_type"])["City"].nunique()

City_region  City_type 
Central      Major          1
             Peripheral    10
East         Major          2
             Peripheral     1
North        Major          1
             Peripheral     6
Northeast    Major          2
             Peripheral     3
South        Major          1
             Peripheral     4
West         Major          1
             Peripheral     2
Name: City, dtype: int64

### Impute missing value

In [14]:
df_no_missing = df.sort_values(by=["City", "time"]).copy()
df_no_missing = df_no_missing.groupby(["City"]).apply(lambda g: g.ffill().bfill(), include_groups=False)
df_no_missing = df_no_missing.reset_index(drop=False)
df_no_missing = df_no_missing.drop(columns=["level_1"])
df_no_missing

,City,Timestamp,name,lat,lon,co,dew,h,no2,o3,p,pm10,pm25,r,so2,t,w,time,City_type,City_region
0,Ayutthaya,2025-02-19 18:11:46,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,-5.9,9.5,3.5,5.2,1008.4,64.0,74.0,0.5,0.6,29.6,2.5,2025-02-20 00:00:00,Peripheral,Central
1,Ayutthaya,2025-02-20 12:00:25,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,23.8,69.5,3.5,5.2,1013.9,64.0,135.0,0.5,0.6,30.0,2.5,2025-02-20 18:00:00,Peripheral,Central
2,Ayutthaya,2025-02-21 01:36:00,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,84.8,3.5,5.2,1011.4,64.0,138.0,0.5,0.6,25.8,1.0,2025-02-21 08:00:00,Peripheral,Central
3,Ayutthaya,2025-02-22 01:53:38,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,76.0,3.5,5.2,1012.9,64.0,95.0,0.5,0.6,27.2,0.5,2025-02-22 08:00:00,Peripheral,Central
4,Ayutthaya,2025-02-22 02:11:00,"Ayutthaya Witthayalai School, Ayutthaya, Thail...",14.352220,100.565325,0.1,25.3,76.0,3.5,5.2,1012.4,64.0,88.0,0.5,0.6,26.6,0.2,2025-02-22 09:00:00,Peripheral,Central
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2075,Yala,2025-04-15 17:33:18,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,25.1,63.0,NaN,NaN,1003.1,33.0,23.0,0.3,NaN,33.1,2.0,2025-04-15 23:00:00,Peripheral,South
2076,Yala,2025-04-17 13:00:19,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,-6.8,7.4,NaN,NaN,1006.5,33.0,27.0,99.2,NaN,32.7,2.0,2025-04-17 19:00:00,Peripheral,South
2077,Yala,2025-04-18 13:00:24,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,-6.5,7.5,NaN,NaN,1006.5,33.0,21.0,99.2,NaN,33.0,2.0,2025-04-18 19:00:00,Peripheral,South
2078,Yala,2025-04-20 02:11:35,"White Elephant Park, Yala, Thailand (สนามโรงพิ...",6.546205,101.283137,NaN,24.0,85.7,NaN,NaN,1008.4,33.0,19.0,99.2,NaN,26.6,1.0,2025-04-20 05:00:00,Peripheral,South


### Resample data to day level

In [15]:
["co", "dew", "h", "no2", "o3", "p", "pm10", "pm25", "r", "so2", "t", "w"]

['co', 'dew', 'h', 'no2', 'o3', 'p', 'pm10', 'pm25', 'r', 'so2', 't', 'w']

In [16]:
df_no_missing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2080 entries, 0 to 2079
Data columns (total 20 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   City         2080 non-null   object        
 1   Timestamp    2080 non-null   object        
 2   name         2080 non-null   object        
 3   lat          2080 non-null   float64       
 4   lon          2080 non-null   float64       
 5   co           1411 non-null   float64       
 6   dew          1837 non-null   float64       
 7   h            2080 non-null   float64       
 8   no2          1654 non-null   float64       
 9   o3           1779 non-null   float64       
 10  p            2080 non-null   float64       
 11  pm10         2017 non-null   float64       
 12  pm25         2080 non-null   float64       
 13  r            1896 non-null   float64       
 14  so2          1653 non-null   float64       
 15  t            2080 non-null   float64       
 16  w     

In [17]:
aqi_pivot = pd.pivot_table(data=df_no_missing, index="time", columns=["City"], values=["pm25", "pm10", "co", "dew", "h", "no2", "o3", "so2", "p", "r", "t", "w"], aggfunc="mean", fill_value=np.nan)
aqi_pivot

co                                                 \
City                Ayutthaya Bangkok Kanchanaburi Lampang Mueang Chiang Rai   
time                                                                           
2025-02-19 12:00:00       NaN     NaN          NaN     NaN               NaN   
2025-02-20 00:00:00       0.1     0.1          0.1     0.1               0.1   
2025-02-20 01:00:00       NaN     NaN          NaN     NaN               NaN   
2025-02-20 18:00:00       0.1     0.1          0.1     0.1               0.1   
2025-02-21 04:00:00       NaN     NaN          NaN     NaN               NaN   
...                       ...     ...          ...     ...               ...   
2025-04-20 08:00:00       NaN     0.1          0.1     NaN               0.1   
2025-04-20 09:00:00       0.1     NaN          NaN     NaN               NaN   
2025-04-21 14:00:00       NaN     NaN          NaN     NaN               NaN   
2025-04-21 19:00:00       NaN     NaN          NaN     NaN               NaN   
2025-04-21 20:00:00       0.1     0.1          0.1     0.1               0.1   

                                                                         ...  \
City                Nakhon Sawan  Nan Nong Khai Nonthaburi Pathum Thani  ...   
time                                                                     ...   
2025-02-19 12:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-02-20 00:00:00          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-20 01:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-02-20 18:00:00          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-21 04:00:00          NaN  NaN       NaN        NaN          NaN  ...   
...                          ...  ...       ...        ...          ...  ...   
2025-04-20 08:00:00          NaN  0.1       0.1        0.1          0.1  ...   
2025-04-20 09:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-04-21 14:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-04-21 19:00:00          NaN  NaN       NaN        NaN          NaN  ...   
2025-04-21 20:00:00          0.1  0.1       0.1        0.1          0.1  ...   

                               w                                          \
City                Samut Prakan Samut Sakhon Saraburi Satun Suphan Buri   
time                                                                       
2025-02-19 12:00:00          NaN          NaN      NaN   NaN         NaN   
2025-02-20 00:00:00          0.5          2.0      0.5   2.0         2.5   
2025-02-20 01:00:00          NaN          NaN      NaN   NaN         NaN   
2025-02-20 18:00:00          1.5          1.0      0.2   3.0         2.0   
2025-02-21 04:00:00          NaN          NaN      NaN   0.5         NaN   
...                          ...          ...      ...   ...         ...   
2025-04-20 08:00:00          1.5          1.5      NaN   NaN         2.0   
2025-04-20 09:00:00          NaN          NaN      0.2   NaN         NaN   
2025-04-21 14:00:00          NaN          NaN      NaN   NaN         NaN   
2025-04-21 19:00:00          NaN          NaN      NaN   1.5         NaN   
2025-04-21 20:00:00          2.5          2.5      0.2   NaN         2.5   

                                                                
City                Trang Trat Ubon Ratchathani Uttaradit Yala  
time                                                            
2025-02-19 12:00:00   NaN  NaN              NaN       1.0  NaN  
2025-02-20 00:00:00   5.1  2.5              1.5       NaN  3.0  
2025-02-20 01:00:00   NaN  NaN              NaN       NaN  NaN  
2025-02-20 18:00:00   1.5  1.0              2.0       3.0  4.6  
2025-02-21 04:00:00   NaN  NaN              NaN       NaN  1.5  
...                   ...  ...              ...       ...  ...  
2025-04-20 08:00:00   0.5  NaN              1.0       1.0  NaN  
2025-04-20 09:00:00   NaN  1.0              NaN       NaN  NaN  
2025-

In [18]:
resampled_aqi = aqi_pivot.resample("D").mean()
resampled_aqi

co                                                 \
City       Ayutthaya Bangkok Kanchanaburi Lampang Mueang Chiang Rai   
time                                                                  
2025-02-19       NaN     NaN          NaN     NaN               NaN   
2025-02-20       0.1     0.1          0.1     0.1               0.1   
2025-02-21       0.1     0.1          0.1     0.1               0.1   
2025-02-22       0.1     0.1          0.1     0.1               0.1   
2025-02-23       0.1     0.1          0.1     0.1               0.1   
...              ...     ...          ...     ...               ...   
2025-04-17       0.1     0.1          0.1     0.1               0.1   
2025-04-18       0.1     0.1          0.1     0.1               0.1   
2025-04-19       NaN     NaN          NaN     NaN               NaN   
2025-04-20       0.1     0.1          0.1     0.1               0.1   
2025-04-21       0.1     0.1          0.1     0.1               0.1   

                                                                ...  \
City       Nakhon Sawan  Nan Nong Khai Nonthaburi Pathum Thani  ...   
time                                                            ...   
2025-02-19          NaN  NaN       NaN        NaN          NaN  ...   
2025-02-20          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-21          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-22          0.1  0.1       0.1        0.1          0.1  ...   
2025-02-23          0.1  0.1       0.1        0.1          0.1  ...   
...                 ...  ...       ...        ...          ...  ...   
2025-04-17          0.1  0.1       0.1        0.1          0.1  ...   
2025-04-18          0.1  0.1       0.1        0.1          0.1  ...   
2025-04-19          NaN  NaN       NaN        NaN          NaN  ...   
2025-04-20          0.1  0.1       0.1        0.1          0.1  ...   
2025-04-21          0.1  0.1       0.1        0.1          0.1  ...   

                      w                                                      \
City       Samut Prakan Samut Sakhon Saraburi Satun Suphan Buri Trang  Trat   
time                                                                          
2025-02-19          NaN          NaN      NaN   NaN         NaN   NaN   NaN   
2025-02-20          1.0          1.5     0.35  2.50    2.250000  3.30  1.75   
2025-02-21          1.0          1.5     0.50  0.50    4.100000  6.10  0.50   
2025-02-22          2.6          2.4     0.60  2.50    1.233333  1.00  1.75   
2025-02-23          2.8          1.5     0.60  4.05    2.300000  2.25  1.00   
...                 ...          ...      ...   ...         ...   ...   ...   
2025-04-17          0.5          1.0     1.00  1.00    1.000000  2.00  2.50   
2025-04-18          2.0          0.5     1.00  1.50    2.000000  1.00  1.00   
2025-04-19          NaN          NaN      NaN   NaN         NaN   NaN   NaN   
2025-04-20          1.5          1.5     0.20  0.50    2.000000  0.50  1.00   
2025-04-21          2.5          2.5     0.20  1.50    2.500000  2.50  1.00   

                                             
City       Ubon Ratchathani Uttaradit  Yala  
time                                         
2025-02-19              NaN       1.0   NaN  
2025-02-20             1.75       3.0  3.80  
2025-02-21             3.60       1.0  1.50  
2025-02-22             3.05       1.0  3.05  
2025-02-23             4.60       2.0  4.30  
...                     ...       ...   ...  
2025-04-17             2.50       3.6  2.00  
2025-04-18             4.10       2.0  2.00  
2025-04-19              NaN       NaN   NaN  
2025-04-20             1.00       1.0  1.00  
2025-04-21             3.00       3.6  1.00  

[62 rows x 370 columns]

In [19]:
resampled_aqi.columns = resampled_aqi.columns.swaplevel(0, 1)
resampled_aqi = resampled_aqi.sort_index(axis=1, level=0)
resampled_aqi

City       Ayutthaya                                                     \
                  co        dew          h  no2   o3            p  pm10   
time                                                                      
2025-02-19       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-02-20       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-21       0.1  25.300000  84.800000  3.5  5.2  1011.400000  64.0   
2025-02-22       0.1  15.066667  53.633333  3.5  5.2  1013.266667  64.0   
2025-02-23       0.1   9.600000  38.700000  3.5  5.2  1013.150000  64.0   
...              ...        ...        ...  ...  ...          ...   ...   
2025-04-17       0.1  -6.500000   7.200000  3.5  5.2  1010.000000  64.0   
2025-04-18       0.1  -5.600000   7.700000  3.5  5.2  1009.700000  64.0   
2025-04-19       NaN        NaN        NaN  NaN  NaN          NaN   NaN   
2025-04-20       0.1  24.700000  80.400000  3.5  5.2  1006.400000  64.0   
2025-04-21       0.1  22.500000  42.000000  3.5  5.2  1008.400000  64.0   

City                               ...  Uttaradit        Yala                  \
                  pm25     r  so2  ...          t    w    dew      h        p   
time                               ...                                          
2025-02-19         NaN   NaN  NaN  ...  23.000000  1.0    NaN    NaN      NaN   
2025-02-20  104.500000   0.5  0.6  ...  26.300000  3.0  -6.65   8.35  1006.55   
2025-02-21  138.000000   0.5  0.6  ...  25.700000  1.0  -8.20   9.20  1009.90   
2025-02-22   82.666667   0.5  0.6  ...  26.266667  1.0   8.30  46.95  1011.20   
2025-02-23   92.500000   0.5  0.6  ...  31.150000  2.0  -7.65   8.30  1009.45   
...                ...   ...  ...  ...        ...  ...    ...    ...      ...   
2025-04-17   76.000000  99.8  0.6  ...  33.400000  3.6  -6.80   7.40  1006.50   
2025-04-18   36.000000  99.6  0.6  ...  32.400000  2.0  -6.50   7.50  1006.50   
2025-04-19         NaN   NaN  NaN  ...        NaN  NaN    NaN    NaN      NaN   
2025-04-20   52.000000  99.6  0.6  ...  29.500000  1.0  24.00  85.70  1008.40   
2025-04-21   52.000000  99.6  0.6  ...  36.300000  3.6  -6.40   7.30  1005.00   

City                                       
            pm10  pm25     r      t     w  
time                                       
2025-02-19   NaN   NaN   NaN    NaN   NaN  
2025-02-20  33.0  44.0   0.2  30.85  3.80  
2025-02-21  33.0  40.0   0.2  27.10  1.50  
2025-02-22  33.0  46.0  99.0  28.90  3.05  
2025-02-23  33.0  44.0  99.0  29.70  4.30  
...          ...   ...   ...    ...   ...  
2025-04-17  33.0  27.0  99.2  32.70  2.00  
2025-04-18  33.0  21.0  99.2  33.00  2.00  
2025-04-19   NaN   NaN   NaN    NaN   NaN  
2025-04-20  33.0  19.0  99.2  26.60  1.00  
2025-04-21  33.0  30.0   0.3  33.50  1.00  

[62 rows x 370 columns]

In [20]:
resampled_aqi = resampled_aqi.ffill().bfill()
resampled_aqi

City       Ayutthaya                                                     \
                  co        dew          h  no2   o3            p  pm10   
time                                                                      
2025-02-19       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-20       0.1   8.950000  39.500000  3.5  5.2  1011.150000  64.0   
2025-02-21       0.1  25.300000  84.800000  3.5  5.2  1011.400000  64.0   
2025-02-22       0.1  15.066667  53.633333  3.5  5.2  1013.266667  64.0   
2025-02-23       0.1   9.600000  38.700000  3.5  5.2  1013.150000  64.0   
...              ...        ...        ...  ...  ...          ...   ...   
2025-04-17       0.1  -6.500000   7.200000  3.5  5.2  1010.000000  64.0   
2025-04-18       0.1  -5.600000   7.700000  3.5  5.2  1009.700000  64.0   
2025-04-19       0.1  -5.600000   7.700000  3.5  5.2  1009.700000  64.0   
2025-04-20       0.1  24.700000  80.400000  3.5  5.2  1006.400000  64.0   
2025-04-21       0.1  22.500000  42.000000  3.5  5.2  1008.400000  64.0   

City                               ...  Uttaradit        Yala                  \
                  pm25     r  so2  ...          t    w    dew      h        p   
time                               ...                                          
2025-02-19  104.500000   0.5  0.6  ...  23.000000  1.0  -6.65   8.35  1006.55   
2025-02-20  104.500000   0.5  0.6  ...  26.300000  3.0  -6.65   8.35  1006.55   
2025-02-21  138.000000   0.5  0.6  ...  25.700000  1.0  -8.20   9.20  1009.90   
2025-02-22   82.666667   0.5  0.6  ...  26.266667  1.0   8.30  46.95  1011.20   
2025-02-23   92.500000   0.5  0.6  ...  31.150000  2.0  -7.65   8.30  1009.45   
...                ...   ...  ...  ...        ...  ...    ...    ...      ...   
2025-04-17   76.000000  99.8  0.6  ...  33.400000  3.6  -6.80   7.40  1006.50   
2025-04-18   36.000000  99.6  0.6  ...  32.400000  2.0  -6.50   7.50  1006.50   
2025-04-19   36.000000  99.6  0.6  ...  32.400000  2.0  -6.50   7.50  1006.50   
2025-04-20   52.000000  99.6  0.6  ...  29.500000  1.0  24.00  85.70  1008.40   
2025-04-21   52.000000  99.6  0.6  ...  36.300000  3.6  -6.40   7.30  1005.00   

City                                       
            pm10  pm25     r      t     w  
time                                       
2025-02-19  33.0  44.0   0.2  30.85  3.80  
2025-02-20  33.0  44.0   0.2  30.85  3.80  
2025-02-21  33.0  40.0   0.2  27.10  1.50  
2025-02-22  33.0  46.0  99.0  28.90  3.05  
2025-02-23  33.0  44.0  99.0  29.70  4.30  
...          ...   ...   ...    ...   ...  
2025-04-17  33.0  27.0  99.2  32.70  2.00  
2025-04-18  33.0  21.0  99.2  33.00  2.00  
2025-04-19  33.0  21.0  99.2  33.00  2.00  
2025-04-20  33.0  19.0  99.2  26.60  1.00  
2025-04-21  33.0  30.0   0.3  33.50  1.00  

[62 rows x 370 columns]

In [21]:
summary_statistics = resampled_aqi.describe().transpose().reset_index()

for metric in summary_statistics["level_1"].unique():
    print(f"Summary statistics of {metric}")
    display(summary_statistics[summary_statistics["level_1"] == metric])

Summary statistics of co


,City,level_1,count,mean,std,min,25%,50%,75%,max
0,Ayutthaya,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
12,Bangkok,co,62.0,0.1,4.197323e-17,0.1,0.1,0.1,0.1,0.1
44,Kanchanaburi,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
54,Lampang,co,62.0,0.1,4.197323e-17,0.1,0.1,0.1,0.1,0.1
77,Mueang Chiang Rai,co,62.0,0.1,4.197323e-17,0.1,0.1,0.1,0.1,0.1
114,Nakhon Sawan,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
126,Nan,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
146,Nong Khai,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
158,Nonthaburi,co,62.0,0.1,4.223569e-17,0.1,0.1,0.1,0.1,0.1
169,Pathum Thani,co,62.0,0.1,4.197323e-17,0.1,0.1,0.1,0.1,0.1


Summary statistics of dew


,City,level_1,count,mean,std,min,25%,50%,75%,max
1,Ayutthaya,dew,62.0,2.340591,12.336350,-8.4,-6.5000,-5.800,14.4000,25.70
33,Chonburi,dew,62.0,4.866935,12.407916,-8.1,-6.3000,-4.250,14.0000,26.60
45,Kanchanaburi,dew,62.0,2.974731,11.564497,-8.2,-5.3375,-4.300,12.5875,24.90
55,Lampang,dew,62.0,2.079032,10.940823,-8.5,-5.1000,-4.300,14.3500,23.50
66,Lamphun,dew,62.0,12.743548,9.633882,-6.5,9.0750,16.700,19.0000,22.50
86,Mueang Khon Kaen,dew,62.0,6.560484,12.172537,-8.2,-5.3750,6.750,18.0000,25.10
94,Nakhon Pathom,dew,62.0,5.533871,13.200130,-8.8,-6.1250,-5.050,20.1500,26.50
104,Nakhon Ratchasima,dew,62.0,3.324194,12.578236,-8.1,-5.1750,-4.400,15.2000,26.50
115,Nakhon Sawan,dew,62.0,4.390323,12.676782,-8.9,-6.0000,-4.500,19.0250,26.80
127,Nan,dew,62.0,3.031452,11.253510,-8.8,-5.4000,-4.800,14.4000,23.90


Summary statistics of h


...

,City,level_1,count,mean,std,min,25%,50%,75%,max
9,Ayutthaya,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
20,Bangkok,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
30,Chiang Mai,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
41,Chonburi,so2,62.0,1.100000,4.477145e-16,1.1,1.1000,1.10,1.1,1.10
63,Lampang,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
74,Lamphun,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
101,Nakhon Pathom,so2,62.0,9.000000,0.000000e+00,9.0,9.0000,9.00,9.0,9.00
111,Nakhon Ratchasima,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
123,Nakhon Sawan,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60
135,Nan,so2,62.0,0.600000,2.238572e-16,0.6,0.6000,0.60,0.6,0.60


Summary statistics of t


,City,level_1,count,mean,std,min,25%,50%,75%,max
10,Ayutthaya,t,62.0,31.756452,2.770771,24.5,30.2125,32.250,33.4000,37.6
21,Bangkok,t,62.0,33.033065,2.290643,26.0,31.8500,33.725,34.5000,36.7
31,Chiang Mai,t,62.0,31.715323,2.889673,24.8,30.1250,32.000,33.8000,36.4
42,Chonburi,t,62.0,31.612097,2.530562,23.7,29.6500,32.700,33.4750,36.2
52,Kanchanaburi,t,62.0,32.681720,3.301648,25.2,30.1500,33.550,35.0000,38.3
64,Lampang,t,62.0,30.531452,3.868528,21.9,27.7250,31.025,33.6250,38.2
75,Lamphun,t,62.0,30.966129,3.502830,22.0,28.7250,31.300,33.8000,38.2
84,Mueang Chiang Rai,t,62.0,30.083065,4.215457,17.7,27.6250,30.800,33.5000,35.7
92,Mueang Khon Kaen,t,62.0,30.740323,4.853416,19.0,26.1000,30.850,35.0000,37.0
102,Nakhon Pathom,t,62.0,31.962903,2.963562,23.3,30.8000,32.725,33.8000,36.0


Summary statistics of w


,City,level_1,count,mean,std,min,25%,50%,75%,max
11,Ayutthaya,w,62.0,2.409677,1.275650,0.2,1.500000,2.500,3.0000,5.10
22,Bangkok,w,62.0,2.145161,1.178625,0.5,1.275000,2.000,3.0000,4.60
32,Chiang Mai,w,62.0,1.447581,0.751361,0.5,1.000000,1.500,2.0000,3.60
43,Chonburi,w,62.0,2.666935,1.336835,0.5,2.000000,2.200,3.0000,6.60
53,Kanchanaburi,w,62.0,1.973387,0.894506,0.5,1.500000,2.000,2.5000,5.10
65,Lampang,w,62.0,1.283871,0.896385,0.5,0.500000,1.000,1.5000,5.00
76,Lamphun,w,62.0,1.897581,0.978432,0.5,1.500000,1.875,2.1125,5.35
85,Mueang Chiang Rai,w,62.0,1.616129,0.884927,0.1,1.000000,1.625,2.0000,3.60
93,Mueang Khon Kaen,w,62.0,1.778226,1.042248,0.5,1.000000,1.500,2.3750,5.60
103,Nakhon Pathom,w,62.0,2.256452,1.016724,0.5,1.500000,2.375,3.0000,4.60


## Hypothesis 1

In [22]:
# Get metadata
city_meta = df_no_missing[['City', 'City_region', 'City_type']].drop_duplicates().set_index("City")

results = []
feature = "pm25"

for region in city_meta["City_region"].unique():
    regional_cities = city_meta[city_meta["City_region"] == region]
    
    majors = regional_cities[regional_cities["City_type"] == "Major"].index
    peripherals = regional_cities[regional_cities["City_type"] == "Peripheral"].index

    for major_city, peripheral_city in product(majors, peripherals):
        try:
            series1 = resampled_aqi[(major_city, feature)].dropna()
            series2 = resampled_aqi[(peripheral_city, feature)].dropna()
        except KeyError:
            continue  # Skip if either city's data is missing

        # Align by common dates
        common_index = series1.index.intersection(series2.index)
        series1 = series1.loc[common_index]
        series2 = series2.loc[common_index]

        if len(series1) < 3 or len(series2) < 3:
            continue

        # Shapiro normality test
        shapiro1 = shapiro(series1)
        shapiro2 = shapiro(series2)

        if shapiro1.pvalue > 0.05 and shapiro2.pvalue > 0.05:
            levene_test = levene(series1, series2)
            ttest = ttest_ind(series1, series2, equal_var=(levene_test.pvalue > 0.05))

            results.append({
                "region": region,
                "major_city": major_city,
                "peripheral_city": peripheral_city,
                "test": "t-test",
                "p_value": ttest.pvalue,
                "shapiro1_p": shapiro1.pvalue,
                "shapiro2_p": shapiro2.pvalue,
                "levene_p": levene_test.pvalue,
            })
        else:
            u_test = mannwhitneyu(series1, series2, alternative="two-sided")
            results.append({
                "region": region,
                "major_city": major_city,
                "peripheral_city": peripheral_city,
                "test": "mannwhitneyu",
                "p_value": u_test.pvalue,
                "shapiro1_p": shapiro1.pvalue,
                "shapiro2_p": shapiro2.pvalue,
                "levene_p": None,
            })

# Wrap in DataFrame
comparison_results = pd.DataFrame(results)
significant_comparisons = comparison_results[comparison_results["p_value"] < 0.05]

In [23]:
comparison_results

,region,major_city,peripheral_city,test,p_value,shapiro1_p,shapiro2_p,levene_p
0,Central,Bangkok,Ayutthaya,mannwhitneyu,1.218165e-02,0.000739,0.066649,NaN
1,Central,Bangkok,Nakhon Pathom,mannwhitneyu,1.426929e-03,0.000739,0.006449,NaN
2,Central,Bangkok,Nakhon Sawan,mannwhitneyu,1.372281e-02,0.000739,0.000260,NaN
3,Central,Bangkok,Nonthaburi,mannwhitneyu,2.364395e-04,0.000739,0.077411,NaN
4,Central,Bangkok,Pathum Thani,mannwhitneyu,4.547958e-03,0.000739,0.007828,NaN
5,Central,Bangkok,Phitsanulok,mannwhitneyu,7.528263e-01,0.000739,0.012013,NaN
6,Central,Bangkok,Samut Prakan,mannwhitneyu,6.348721e-01,0.000739,0.002519,NaN
7,Central,Bangkok,Samut Sakhon,mannwhitneyu,1.515530e-05,0.000739,0.006973,NaN
8,Central,Bangkok,Saraburi,mannwhitneyu,9.353249e-06,0.000739,0.003264,NaN
9,Central,Bangkok,Suphan Buri,mannwhitneyu,1.661274e-07,0.000739,0.000003,NaN


In [24]:
significant_comparisons

,region,major_city,peripheral_city,test,p_value,shapiro1_p,shapiro2_p,levene_p
0,Central,Bangkok,Ayutthaya,mannwhitneyu,1.218165e-02,0.000739,0.066649,NaN
1,Central,Bangkok,Nakhon Pathom,mannwhitneyu,1.426929e-03,0.000739,0.006449,NaN
2,Central,Bangkok,Nakhon Sawan,mannwhitneyu,1.372281e-02,0.000739,0.000260,NaN
3,Central,Bangkok,Nonthaburi,mannwhitneyu,2.364395e-04,0.000739,0.077411,NaN
4,Central,Bangkok,Pathum Thani,mannwhitneyu,4.547958e-03,0.000739,0.007828,NaN
7,Central,Bangkok,Samut Sakhon,mannwhitneyu,1.515530e-05,0.000739,0.006973,NaN
8,Central,Bangkok,Saraburi,mannwhitneyu,9.353249e-06,0.000739,0.003264,NaN
9,Central,Bangkok,Suphan Buri,mannwhitneyu,1.661274e-07,0.000739,0.000003,NaN
10,North,Chiang Mai,Lampang,mannwhitneyu,6.826784e-04,0.008207,0.016125,NaN
11,North,Chiang Mai,Lamphun,mannwhitneyu,4.634405e-03,0.008207,0.007953,NaN


In [ ]:
# export train data for modeling
unstack_resampled = resampled_aqi.unstack().reset_index()
train_data = unstack_resampled.pivot_table(values=0, index=["City", "time"], columns=["level_1"]).reset_index()
# train_data.to_csv("air_data_train.csv")

In [ ]:
# export test data (19 - 21 April, 2025)
unstack_resampled = resampled_aqi.unstack().reset_index()
test_data = unstack_resampled.pivot_table(values=0, index=["City", "time"], columns=["level_1"]).reset_index()
test_dates = pd.to_datetime(["2025-04-19", "2025-04-20", "2025-04-21"])
test_data = test_data[test_data["time"].isin(test_dates)]
# test_data.to_csv("air_quality_test.csv")